In [ ]:
#ensure your json files are present.
import os
print(os.listdir())


In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
#Initial Installiations
!pip install gspread oauth2client --quiet
!pip install -U langchain-openai --quiet
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet
!pip install --upgrade google-auth google-auth-oauthlib google-api-python-client


In [ ]:
#Imports
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_openai import ChatOpenAI
import os
import json
from typing import List, Dict
from getpass import getpass
from datetime import datetime
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64


In [ ]:
# STEP 2: Set OpenRouter API Key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenRouter API Key: ")

# STEP 3: Load OpenRouter model (Mistral via OpenRouter)
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.5,
)

In [ ]:
#GOOGLE SHEETS INTEGRATION
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 1. Define scopes
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

# 2. Load credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(
    "projectemailsorter-62df267deeed.json", scope
)

# 3. Authorize client
client = gspread.authorize(creds)

# 4. Try to list available spreadsheets to verify access
sheets = client.openall()
print("✅ Accessible Sheets:")
for sheet in sheets:
    print("-", sheet.title)
    
sheet = client.open("LangChain").sheet1
sheet.update("A1", [["✅ Connected successfully!"]])
print("🎉 Sheet updated!")


In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    from google_auth_oauthlib.flow import InstalledAppFlow
    from google.auth.transport.requests import Request
    from googleapiclient.discovery import build
    from google.oauth2.credentials import Credentials
    import os

    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    creds = None

    # Load saved credentials if available
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # If not valid, trigger authentication
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            
            flow = InstalledAppFlow.from_client_secrets_file('client_secret2.json', SCOPES)
            creds = flow.run_local_server(port=0)  

        # Save credentials for next time
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

def get_unread_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread", maxResults=max_results).execute()
    messages = results.get('messages', [])
    emails = []

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
        headers = msg_data['payload']['headers']
        subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
        body = ""

        if 'parts' in msg_data['payload']:
            for part in msg_data['payload']['parts']:
                if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                    body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
                    break

        emails.append({"subject": subject, "body": body.strip()})

    return emails


# 🔍 Step 4: LangChain Tool + Prompt
@tool
def parse_user_request(text: str) -> dict:
    """Classify a user query into structured metadata (summary, type, is_junk)."""
    text = text.lower()
    if "refund" in text or "replace" in text:
        return {"summary": "Request for refund or replacement.", "type": "refund_request", "is_junk": False}
    elif "hello" in text or "test" in text:
        return {"summary": "Greeting or test message.", "type": "test", "is_junk": True}
    elif not text.strip():
        return {"summary": "Empty input.", "type": "unknown", "is_junk": True}
    else:
        return {"summary": "Unclassified request.", "type": "unknown", "is_junk": True}


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant classifying customer support queries."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm
)

# 📝 Step 5: Sheet Logger
def log_to_sheet(data: Dict):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sheet.append_row([
        now,
        data.get("input", ""),
        data.get("summary", ""),
        data.get("type", ""),
        "Yes" if data.get("is_junk") else "No"
    ])


# 🤖 Step 6: Agent Executor
class SupportAgentExecutor:
    def __init__(self):
        self.chat_history = []
        self.agent = agent  # agent must be your LangChain RunnableSerializable

    def invoke(self, user_input: str) -> dict:
        print("⏳ Asking LLM...")
        result = self.agent.invoke({
            "input": user_input,
            "chat_history": self.chat_history,
            "agent_scratchpad": []
        })

        self.chat_history.extend([
            HumanMessage(content=user_input),
            AIMessage(content=result.content)
        ])

        print("⏳ Parsing user input...")
        parsed = parse_user_request.invoke(result.content)

        # log_to_sheet must be defined
        log_to_sheet({
            "input": user_input,
            "summary": parsed["summary"],
            "type": parsed["type"],
            "is_junk": parsed["is_junk"]
        })

        return parsed

# ✅ Create the executor instance
executor = SupportAgentExecutor()



In [ ]:
gmail_service = authenticate_gmail()
labels = gmail_service.users().labels().list(userId='me').execute()
print("📬 Gmail Labels:")
for label in labels['labels']:
    print("-", label['name'])


In [ ]:
emails = gmail_service.users().messages().list(
    userId='me',
    labelIds=['INBOX'],
    q='is:unread',
    maxResults=5
).execute()

messages = emails.get('messages', [])

print(f"📨 Unread Messages: {len(messages)}")

for msg in messages:
    msg_data = gmail_service.users().messages().get(userId='me', id=msg['id']).execute()
    headers = msg_data['payload']['headers']
    subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')
    print("🧾 Subject:", subject)

In [ ]:
from base64 import urlsafe_b64decode

emails = gmail_service.users().messages().list(
    userId='me',
    labelIds=['INBOX'],
    q='is:unread',
    maxResults=5
).execute()

messages = emails.get('messages', [])
print(f"📨 Unread Messages: {len(messages)}")

for msg in messages:
    msg_data = gmail_service.users().messages().get(userId='me', id=msg['id']).execute()
    headers = msg_data['payload']['headers']
    subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')

    # Get plain text body
    parts = msg_data['payload'].get('parts', [])
    body = ""
    for part in parts:
        if part['mimeType'] == 'text/plain' and 'data' in part['body']:
            body = urlsafe_b64decode(part['body']['data']).decode('utf-8')
            break

    print(f"\n📬 Subject: {subject}")
    print("🧠 Classifying...")

    # Run through your AI agent
    result = executor.invoke(body)

    print("✅ Classified:", result)
